In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/wsdm-kkbox/songs.csv
/kaggle/input/wsdm-kkbox/members.csv
/kaggle/input/wsdm-kkbox/train.csv
/kaggle/input/wsdm-kkbox/test.csv
/kaggle/input/wsdm-kkbox/song_extra_info.csv


In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from ast import literal_eval
from surprise import SVD, SVDpp, KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate,train_test_split, GridSearchCV
from surprise import NormalPredictor
from surprise import Reader

import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings.csv')

In [16]:
ratings_array = ratings['rating'].unique()
max_rating = np.amax( ratings_array )
min_rating = np.amin( ratings_array )
print( ratings_array )

[1.  4.5 5.  4.  3.5 2.5 0.5 3.  2.  1.5]


In [ ]:
m_data = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')

In [ ]:
m_data['genres'] = m_data['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
m_data['genres_str']= m_data['genres'].apply(lambda x: ",".join(x))

In [ ]:
m_data.head(5)

In [17]:
def get_movieId( movie_name ):
    """
    return the movieId which is corresponding to the movie name

    Parameters
    ----------
    movie_name: string, the name of the movie w/ or w/o the year

    Return
    ------
    the movieId
    """
    # If luckily the movie name is 100% equal to a name writen in the database,
    # then return the id corresponding to the name.
    # Or...we need to consider the similarity between strings 
    if (movie_name in movie_map):
        return movie_map[movie_name]
    else:
        similar = []
        for title, movie_id in movie_map.items():
            ratio = fuzz.ratio(title.lower(), movie_name.lower())
            if ( ratio >= 60):
                similar.append( (title, movie_id, ratio ) )
        if (len(similar) == 0):
            print("Oh! This movie does not exist in the database.")
        else:
            match_item = sorted( similar , key=lambda x: x[2] )[::-1]
            print( "The matched item might be:", match_item[0][0], ", ratio=",match_item[0][2] )
            return match_item[0][1]

In [18]:
def tokenizer(text):
    torkenized = [PorterStemmer().stem(word).lower() for word in text.split('|') if word not in stopwords.words('english')]
    return torkenized

In [ ]:
tfid=TfidfVectorizer(analyzer='word', tokenizer=tokenizer)

In [ ]:
tfidf_matrix = tfid.fit_transform(m_data['genres_str'])

In [ ]:
cos_sim = cosine_similarity(tfidf_matrix,tfidf_matrix)

In [ ]:
cos_sim

In [ ]:
print(tfidf_matrix.shape)
print(cos_sim.shape)
print(m_data.shape)

In [ ]:
m_data

In [ ]:
ratings

In [19]:
features = ['userId','movieId', 'rating']
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings[features], reader)
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25)


In [20]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
#gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
svd = SVD()
#consume too much memory
#cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [21]:
# gs.fit(data)
# print(gs.best_score['rmse'])
# print(gs.best_params['rmse'])
# best_params = gs.best_params['rmse']
# model_svd = gs.best_estimator['rmse']
# #没有做数据集划分
# model_svd.fit(data.build_full_trainset())

In [22]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25)

In [23]:
def get_rating_from_prediction( prediction, ratings_array ):
    """
    return the closest rating number to the prediction value

    Parameters
    ----------
    prediction: float, the prediction value from the model

    ratings_array: the 1D array of the discrete rating number

    Return
    ------
    the rating number corresponding to the prediction value
    """
    rating = ratings_array[ np.argmin( [ np.abs(item - prediction) for item in ratings_array ] ) ]
    return rating

In [24]:
trainset = data.build_full_trainset()
svd.fit(trainset)


In [27]:
predictions = svd.test(testset)

In [29]:
#del trainset
del testset

In [30]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 0.6791


0.6790566411936375

In [ ]:
from surprise.model_selection import KFold


kf = KFold(n_splits=5)

algo = SVD()

for trainset, testset in kf.split(data):

    # 训练并测试算法
    algo.fit(trainset)
    predictions = algo.test(testset)

    # 计算并打印 RMSE（均方根误差，Root Mean Squared Error）
    accuracy.rmse(predictions, verbose=True)